# OCaml

Principi programskih jezikov, 2024-04-17

---

## Signatura in implementacija grup

In [1]:
1 + 1

- : int = 2


In [2]:
module Z3 =
struct

  type g = Zero | One | Two

  let e = Zero

  let plus = function
    | (Zero, x) | (x, Zero) -> x
    | (One, One) -> Two
    | (One, Two) | (Two, One) -> Zero
    | (Two, Two) -> One

  let mul = plus

  let inv = function
    | Zero -> Zero
    | One -> Two
    | Two -> One
end

module Z3 :
  sig
    type g = Zero | One | Two
    val e : g
    val plus : g * g -> g
    val mul : g * g -> g
    val inv : g -> g
  end


In [5]:
Z3.mul (Z3.e, (Z3.inv Z3.One))

- : Z3.g = Z3.Two


In [6]:
module type GROUP =
sig
    type g
    val mul : g * g -> g
    val inv : g -> g
    val e : g
end

module type GROUP =
  sig type g val mul : g * g -> g val inv : g -> g val e : g end


In [9]:
module Z3 : GROUP =
struct

  type g = Zero | One | Two

  let e = Zero

  let plus = function
    | (Zero, y) -> y
    | (x, Zero) -> x
    | (One, One) -> Two
    | (One, Two) -> Zero
    | (Two, One) -> Zero
    | (Two, Two) -> One

  let mul = plus

  let inv = function
    | Zero -> Zero
    | One -> Two
    | Two -> One
end

module Z3 : GROUP


In [15]:
Z3.mul (Z3.e, (Z3.inv Z3.e))

- : Z3.g = <abstr>


In [ ]:
module Z3' : GROUP =
struct
    type g = int

    let mul (x, y) = (x + y) mod 3
    let inv x = (3 - x) mod 3
    let e = 0
end

## Prednostne vrste

In [33]:
module type PRIORITY_QUEUE =
  sig
    type element
    val priority : element -> int

    type queue
    val empty : queue
    val put : element -> queue -> queue
    val get : queue -> element option * queue
  end

module type PRIORITY_QUEUE =
  sig
    type element
    val priority : element -> int
    type queue
    val empty : queue
    val put : element -> queue -> queue
    val get : queue -> element option * queue
  end


### Naivna implementacija prioritetne vrste

In [34]:
module MyFirstQueue : PRIORITY_QUEUE with type element = int * int =
  struct
    type element = int * int

    let priority (a, b) = a

    type queue = element list

    let empty = []

    let rec put x = function
      | [] -> [x]
      | y :: ys ->
         if priority x <= priority y then
           x :: y :: ys
         else
           y :: put x ys

    let get = function
      | [] -> (None, [])
      | x :: xs -> (Some x, xs)
  end

module MyFirstQueue :
  sig
    type element = int * int
    val priority : element -> int
    type queue
    val empty : queue
    val put : element -> queue -> queue
    val get : queue -> element option * queue
  end


In [35]:
MyFirstQueue.empty
|> MyFirstQueue.put (100, 123)
|> MyFirstQueue.put (1, 456)
|> MyFirstQueue.put (20, 789)
|> MyFirstQueue.get

- : MyFirstQueue.element option * MyFirstQueue.queue =
(Some (1, 456), <abstr>)


In [24]:
MyFirstQueue.empty
|> MyFirstQueue.put (100, 123)
|> MyFirstQueue.put (1, 456)
|> MyFirstQueue.put (20, 789)
|> MyFirstQueue.get

- : MyFirstQueue.element option * MyFirstQueue.queue =
(Some (1, 456), <abstr>)


In [36]:
module MyFirstStringQueue : PRIORITY_QUEUE with type element = string =
  struct
    type element = string

    let priority = String.length

    type queue = element list

    let empty = []

    let rec put x = function
      | [] -> [x]
      | y :: ys ->
         if priority x <= priority y then
           x :: y :: ys
         else
           y :: put x ys

    let get = function
      | [] -> (None, [])
      | x :: xs -> (Some x, xs)
  end

module MyFirstStringQueue :
  sig
    type element = string
    val priority : element -> int
    type queue
    val empty : queue
    val put : element -> queue -> queue
    val get : queue -> element option * queue
  end


In [37]:
MyFirstStringQueue.empty
|> MyFirstStringQueue.put "banana"
|> MyFirstStringQueue.put "jabolko"
|> MyFirstStringQueue.put "kivi"
|> MyFirstStringQueue.get

- : MyFirstStringQueue.element option * MyFirstStringQueue.queue =
(Some "kivi", <abstr>)


## Funktorji

In [39]:
module type PRIORITY_TYPE =
sig
  type t
  val priority : t -> int
end

module type PRIORITY_TYPE = sig type t val priority : t -> int end


In [47]:
module StringPriority : PRIORITY_TYPE with type t = string =
struct
  type t = string
  let priority s = String.length s
end

module StringPriority : sig type t = string val priority : t -> int end


In [51]:
module ListQueue (P : PRIORITY_TYPE) : PRIORITY_QUEUE with type element = P.t =
  struct
    type element = P.t

    let priority = P.priority

    type queue = element list

    let empty = []

    let rec put x = function
      | [] -> [x]
      | y :: ys ->
         if priority x <= priority y then
           x :: y :: ys
         else
           y :: put x ys

    let get = function
      | [] -> (None, [])
      | x :: xs -> (Some x, xs)
  end

module ListQueue :
  functor (P : PRIORITY_TYPE) ->
    sig
      type element = P.t
      val priority : element -> int
      type queue
      val empty : queue
      val put : element -> queue -> queue
      val get : queue -> element option * queue
    end


In [52]:
module StringListQueue = ListQueue(StringPriority)

module StringListQueue :
  sig
    type element = StringPriority.t
    val priority : element -> int
    type queue = ListQueue(StringPriority).queue
    val empty : queue
    val put : element -> queue -> queue
    val get : queue -> element option * queue
  end


In [53]:
StringListQueue.empty
|> StringListQueue.put "banana"
|> StringListQueue.put "jabolko"
|> StringListQueue.put "kivi"
|> StringListQueue.get

- : StringListQueue.element option * StringListQueue.queue =
(Some "kivi", <abstr>)


In [54]:
module StringListQueue' =
  ListQueue(
      struct
        type t = string
        let priority = String.length
      end)

module StringListQueue' :
  sig
    type element = string
    val priority : element -> int
    type queue
    val empty : queue
    val put : element -> queue -> queue
    val get : queue -> element option * queue
  end


In [55]:
StringListQueue'.empty
|> StringListQueue'.put "banana"
|> StringListQueue'.put "jabolko"
|> StringListQueue'.put "kivi"
|> StringListQueue'.get

- : StringListQueue'.element option * StringListQueue'.queue =
(Some "kivi", <abstr>)


## Učinkovitejša implementacija

Učinkovita implementacija z [levičarskimi kopicami](<https://en.wikipedia.org/wiki/Leftist_tree>). Implementacija je abstraktna, ker uporabimo `:`, vendar dodamo določilo, da je tip `element` enak tipu `t` iz modula `M`.

In [56]:
module LeftistHeapQueue (P : PRIORITY_TYPE) : PRIORITY_QUEUE with type element = P.t =
  struct
    type element = P.t

    let priority = P.priority

    type queue = Leaf | Node of int * element * queue * queue

    let rank = function
    | Leaf -> 0
    | Node (r, _, _, _) -> r

    let node (x, a, b) =
    if rank a < rank b then
      Node (1 + rank a, x, b, a)
    else
      Node (1 + rank b, x, a, b)

    let rec meld a b =
    match (a, b) with
    | (_, Leaf) -> a
    | (Leaf, _) -> b
    | (Node (_, ka, la, ra), Node (_, kb, lb, rb)) ->
      if priority ka < priority kb then
        node (ka, la, meld ra b)
      else
        node (kb, lb, meld a rb)

    let singleton x = Node (1, x, Leaf, Leaf)

    let empty = Leaf

    let put x q = meld q (singleton x)

    let get = function
    | Leaf -> (None, Leaf)
    | Node (_, y, l, r) -> (Some y, meld l r)
  end

module LeftistHeapQueue :
  functor (P : PRIORITY_TYPE) ->
    sig
      type element = P.t
      val priority : element -> int
      type queue
      val empty : queue
      val put : element -> queue -> queue
      val get : queue -> element option * queue
    end


In [57]:
module Fast = LeftistHeapQueue(
               struct
                 type t = int * int
                 let priority (x, y) = x + y
               end)


module Fast :
  sig
    type element = int * int
    val priority : element -> int
    type queue
    val empty : queue
    val put : element -> queue -> queue
    val get : queue -> element option * queue
  end


In [58]:
module Slow = ListQueue(
               struct
                 type t = int * int
                 let priority (x, y) = x + y
               end)

module Slow :
  sig
    type element = int * int
    val priority : element -> int
    type queue
    val empty : queue
    val put : element -> queue -> queue
    val get : queue -> element option * queue
  end


In [59]:
let rec loop q = function
  | 0 -> Slow.put (0, 0) q
  | k -> loop (Slow.put ((47 * k * k + 13) mod 1000, k) q) (k - 1)
in
loop Slow.empty 300000

- : Slow.queue = <abstr>


In [60]:
let rec loop q = function
  | 0 -> Fast.put (0, 0) q
  | k -> loop (Fast.put ((47 * k * k + 13) mod 1000, k) q) (k - 1)
in
loop Fast.empty 300000

- : Fast.queue = <abstr>
